# AI Underwriter Assistant

Author: https://github.com/deburky

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')

## Prepare training data

Loan approval dataset is used to train the model with a few-shot learning.

In [2]:
# Fetch loan approval data
import pandas as pd
url = (
    "https://drive.google.com/file/d/1r6gGvL_s313ThGSU7ziZiuYr2G_yijaZ/view?usp=sharing"
)
file_id = url.split("/")[-2]
dataset = pd.read_csv(f"https://drive.google.com/uc?id={file_id}")

dataset.drop(columns=['Gender']).sample(n=700).to_csv('loans-9309cbc146a4.csv', index=False)
dataset.drop(columns=['Gender', 'Loan_Status']).sample(n=100).to_csv('loans-new-9309cbc146a4.csv', index=False)

train_set = pd.read_csv("loans-new-9309cbc146a4.csv")
display(train_set.sample(5))

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
22,LP001497,No,0,Graduate,Yes,4895,0.0,94.0,360.0,1.0,Semiurban
54,LP001978,No,0,Graduate,No,4692,4333.0,162.0,300.0,1.0,Urban
48,LP001574,Yes,2,Not Graduate,Yes,3917,0.0,124.0,360.0,1.0,Urban
66,LP002129,Yes,2,Graduate,NaN,3333,461.0,108.0,360.0,1.0,Urban
8,LP002531,No,0,Graduate,Yes,2900,0.0,100.0,360.0,1.0,Rural


## Assistant

[OpenAI - Assistants API quickstart](https://platform.openai.com/docs/assistants/quickstart)

[Assistants API deep dive](https://platform.openai.com/docs/assistants/deep-dive)

[Fun with Assistants – getting them to really talk to each other](https://community.openai.com/t/fun-with-assistants-getting-them-to-really-talk-to-each-other/784073/12)

In [5]:
from openai import OpenAI
client = OpenAI()

# Upload a file with loans data
loan_file = client.files.create(
    file=open("loans-9309cbc146a4.csv", "rb"),
    purpose='assistants'
)

loan_file_new = client.files.create(
    file=open("loans-new-9309cbc146a4.csv", "rb"),
    purpose='assistants'
)

# Create an assistant for AI underwriting
assistant = client.beta.assistants.create(
    name="AI Underwriter Assistant",
    description="""
        AI Underwriter Assistant specialized in underwriting loan applications.
        """
    ,
    model="gpt-4o-mini",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
        "file_ids": [loan_file.id, loan_file_new.id]
        }
    }
)

## Define the prompt for evaluation

In [6]:
# Define the prompt
prompt = """
    I want you to first familiarize yourself with the past decisions in the first file.
    After understanding the logic of past decisions, I want you to evaluate the decisions in the second file.
    Your job is to give an answer "Y" (yes) or "N" (no) for each decision in the second file.
    Like: Application 1. Decision: Yes. And so on.
    In the end, please prepare a csv with 'Loan_ID' and 'Loan_Status' columns that you fill in for the second file.
    In the file your answers must be recorded as 'Y' or 'N'.
"""

# Create a thread with the prompt
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

# Run the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [7]:
import time
from rich.console import Console
from rich.live import Live
from rich.table import Table

console = Console()

console.print(f"🔄 [cyan1]Run started... (Run ID: {run.id})[/cyan1]")

# Function to fetch messages and display them
def fetch_messages():
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    table = Table(title="Assistant Conversation", show_header=True, header_style="bold cyan1")
    table.add_column("Role", style="cyan1", width=12)
    table.add_column("Message", style="white")

    for message in reversed(messages.data):  # Reverse to show latest first
        text = message.content[0].text.value if message.content else "[No content]"
        table.add_row(message.role, text)

    return table

# Monitor the run until completion
with Live(fetch_messages(), refresh_per_second=1, console=console) as live:
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status in ["completed", "failed", "cancelled"]:
            break
        live.update(fetch_messages())
        time.sleep(2)

# Final message update
console.print(fetch_messages())
console.print("✅ [cyan1]Run completed![/cyan1]")

🔄 Run started... (Run ID: run_tHWYOPO25sNyBtVOlXV2z2qJ)

/Users/deburky/Library/Caches/pypoetry/virtualenvs/language-models-qT78hDLD-py3.10/lib/python3.10/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

                                              Assistant Conversation                                               
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Role         ┃ Message                                                                                          ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user         │                                                                                                  │
│              │     I want you to first familiarize yourself with the past decisions in the first file.          │
│              │     After understanding the logic of past decisions, I want you to evaluate the decisions in the │
│              │ second file.                                                                                     │
│              │     Your job is to give an answer "Y" (yes) or "N" (no) for each decision in the second file.    │
│              │     Like: Application 1. Decision: Yes. And so on.                                               │
│              │     In the end, please prepare a csv with 'Loan_ID' and 'Loan_Status' columns that you fill in   │
│              │ for the second file.                                                                             │
│              │     In the file your answers must be recorded as 'Y' or 'N'.                                     │
│              │                                                                                                  │
│ assistant    │ Let's start by reviewing the contents of the first file to understand the past decisions made.   │
│              │ After that, I will analyze the second file based on that understanding. I'll begin by loading    │
│              │ and examining the contents of the first file.                                                    │
│ assistant    │ The first file appears to be a CSV containing information about various loan applications and    │
│              │ their respective features. Here are the relevant columns based on the initial examination:       │
│              │                                                                                                  │
│              │ - **Loan_ID**: Identifier for the loan application                                               │
│              │ - **Married**: Marital status of the applicant                                                   │
│              │ - **Dependents**: Number of dependents                                                           │
│              │ - **Education**: Education level of the applicant                                                │
│              │ - **Self_Employed**: Indicates if the applicant is self-employed                                 │
│              │ - **ApplicantIncome**: Income of the applicant                                                   │
│              │ - **CoapplicantIncome**: Income of the coapplicant (if any)                                      │
│              │ - **LoanAmount**: Amount of loan requested                                                       │
│              │ - **Loan_Amount_Term**: Duration of the loan                                                     │
│              │ - **Credit_History**: Indicates if the applicant has a good credit history (1) or not (0)        │
│              │ - **Property_Area**: Area classification of the property (Urban, Semiurban, Rural)               │
│              │                                                                                                  │
│              │ Next, I will split this data into a structured format to better understand the decisions made    │
│              │ (likely implied or based on some outcome not shown in this snippet). I'll check if any decision  │
│              │ column exists and look into it.        

✅ Run completed!

## Save report

In [8]:
import pandas as pd
from openai import OpenAI

# Fetch the list of files
files = client.files.list()

# Extract filenames and file IDs
file_info = [(file.id, file.filename, file.purpose, file.status) for file in files.data]

# Convert to DataFrame for better readability
df_files = pd.DataFrame(file_info, columns=["File ID", "Filename", "Purpose", "Status"])

# # Display the extracted filenames
display(df_files)

,File ID,Filename,Purpose,Status
0,file-U81soHHm8YAw9Qy8ndXfXZ,/mnt/data/loan_evaluation_output.csv,assistants_output,processed
1,file-LfA8WbcrjLM4xQyCKjz6cp,loans-new-9309cbc146a4.csv,assistants,processed
2,file-3qo88HgvYko34q9xiqjMJy,loans-9309cbc146a4.csv,assistants,processed


In [9]:
import io
evaluation_results = df_files.iloc[0]['File ID'] # Fetch assistants_output
content = client.files.retrieve_content(evaluation_results)
csv_string = content
csv_file = io.StringIO(csv_string)
df = pd.read_csv(csv_file)
print(df.head(5))

/var/folders/k_/yz8rvp25185_js60dw8vhnj40000gn/T/ipykernel_70962/3349831283.py:3: DeprecationWarning: The `.content()` method should be used instead
  content = client.files.retrieve_content(evaluation_results)


    Loan_ID Loan_Status
0  LP002556           Y
1  LP002840           Y
2  LP001750           Y
3  LP002141           Y
4  LP001715           N


In [10]:
df.to_csv('loans-new-FKJESKpN5LYuSG21np1PYE.csv', index=False)

## Evaluation against ground truth

In [11]:
# Get real decisions outcomes
testing_df = df.merge(
    dataset[['Loan_ID', 'Loan_Status']], 
    on='Loan_ID',
    suffixes=('_Assistant', '_Truth')
)
testing_df.replace({'Y': 1, 'N': 0}, inplace=True)

/var/folders/k_/yz8rvp25185_js60dw8vhnj40000gn/T/ipykernel_70962/2525230116.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_df.replace({'Y': 1, 'N': 0}, inplace=True)


In [12]:
display(testing_df[['Loan_Status_Assistant', 'Loan_Status_Truth']].mean())
decision_matrix = pd.crosstab(testing_df['Loan_Status_Truth'], testing_df['Loan_Status_Assistant'], normalize='all')
display(decision_matrix)

Loan_Status_Assistant    0.847143
Loan_Status_Truth        0.706336
dtype: float64

Loan_Status_Assistant,0,1
Loan_Status_Truth,,
0,0.049689,0.243975
1,0.103168,0.603168


## Delete the assistant and files

In [13]:
# Fetch all assistants
assistants = client.beta.assistants.list()

# Delete each assistant
for assistant in assistants.data:
    print(f"Deleting Assistant: {assistant.name} (ID: {assistant.id})")
    client.beta.assistants.delete(assistant.id)

print("✅ All assistants deleted!")

# Fetch all files
files = client.files.list()

# Delete each file
for file in files.data:
    print(f"Deleting File: {file.filename} (ID: {file.id})")
    client.files.delete(file.id)

print("✅ All files deleted!")

Deleting Assistant: AI Underwriter Assistant (ID: asst_lJPGtnGNlxWeK1rBkkOTkdEK)
✅ All assistants deleted!
Deleting File: /mnt/data/loan_evaluation_output.csv (ID: file-U81soHHm8YAw9Qy8ndXfXZ)
Deleting File: loans-new-9309cbc146a4.csv (ID: file-LfA8WbcrjLM4xQyCKjz6cp)
Deleting File: loans-9309cbc146a4.csv (ID: file-3qo88HgvYko34q9xiqjMJy)
✅ All files deleted!
